# Notebook para transformar datos desde una base SQLite 'sucia' a un CSV limpio con pandas, incluyendo conversiones de tipos y fusiones.
Este cuaderno reproduce, paso a paso, el flujo ETL desde `data.sqlite` hasta `data.csv`, destacando CAST/COERCE de tipos, manejo de NaN (FILLNA/DROPNA) y MERGEs/joins clave.

# 1) Conectar a SQLite y listar tablas
En esta sección abrimos la base de datos y listamos tablas disponibles con sus conteos de filas.

In [2]:
# Imports y rutas
import os, sqlite3, json, textwrap, unicodedata, re
from pathlib import Path
import numpy as np
import pandas as pd

DATA_DIR = Path(r"C:/Users/Joaquim/OneDrive/UPGRADE/PROYECTO1_Soccer/data")
DB_PATH = DATA_DIR/"data.sqlite"
CSV_PATH = DATA_DIR/"data.csv"

print("DB exists?", DB_PATH.exists(), "->", DB_PATH)
print("CSV target:", CSV_PATH)

DB exists? True -> C:\Users\Joaquim\OneDrive\UPGRADE\PROYECTO1_Soccer\data\data.sqlite
CSV target: C:\Users\Joaquim\OneDrive\UPGRADE\PROYECTO1_Soccer\data\data.csv


In [2]:
# Listar tablas y conteos
import pandas as pd
import sqlite3

with sqlite3.connect(DB_PATH) as conn:
    tablas = pd.read_sql("SELECT name, type FROM sqlite_master WHERE type IN ('table','view') ORDER BY name", conn)
    counts = []
    for name in tablas['name']:
        try:
            c = pd.read_sql(f"SELECT COUNT(*) as n FROM '{name}'", conn)['n'].iloc[0]
        except Exception:
            c = None
        counts.append(c)
    tablas['rows'] = counts

tablas

,name,type,rows
0,Country,table,11
1,League,table,11
2,Match,table,25979
3,Player,table,11060
4,Player_Attributes,table,183978
5,Team,table,299
6,Team_Attributes,table,1458
7,sqlite_sequence,table,7


# 2) Cargar tablas a DataFrames y vista previa con head()
Cargamos únicamente las columnas necesarias para optimizar lectura.

In [3]:
from sqlalchemy import create_engine
from pandas import read_sql

COLS_PLAYER = "player_api_id, player_name, birthday, height, weight"
COLS_ATTR = ("player_api_id, date, overall_rating, preferred_foot, attacking_work_rate, defensive_work_rate, "
             "ball_control, dribbling, finishing, free_kick_accuracy, heading_accuracy, short_passing, shot_power, "
             "penalties, acceleration, sprint_speed, agility, stamina, jumping, aggression, gk_diving, gk_reflexes")
COLS_MATCH = ("date, match_api_id, home_team_api_id, away_team_api_id, league_id, "
              "home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, "
              "away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11")
COLS_TEAM = "team_api_id, team_long_name"
COLS_LEAGUE = "id, country_id, name"
COLS_COUNTRY = "id, name"

engine = create_engine(f"sqlite:///{DB_PATH}")

player = read_sql(f'SELECT {COLS_PLAYER} FROM Player', engine)
attr = read_sql(f'SELECT {COLS_ATTR} FROM Player_Attributes', engine)
match = read_sql(f'SELECT {COLS_MATCH} FROM Match', engine)
team = read_sql(f'SELECT {COLS_TEAM} FROM Team', engine)
league = read_sql(f'SELECT {COLS_LEAGUE} FROM League', engine).rename(columns={'name':'league_name'})
country = read_sql(f'SELECT {COLS_COUNTRY} FROM Country', engine).rename(columns={'name':'country_name'})

print('player', player.shape)
print('attr', attr.shape)
print('match', match.shape)
print('team', team.shape)
print('league', league.shape)
print('country', country.shape)

player.head(), attr.head(), match.head(), team.head(), league.head(), country.head()

player (11060, 5)
attr (183978, 22)
match (25979, 27)
team (299, 2)
league (11, 3)
country (11, 2)


(   player_api_id         player_name             birthday  height  weight
 0         505942  Aaron Appindangoye  1992-02-29 00:00:00  182.88     187
 1         155782     Aaron Cresswell  1989-12-15 00:00:00  170.18     146
 2         162549         Aaron Doran  1991-05-13 00:00:00  170.18     163
 3          30572       Aaron Galindo  1982-05-08 00:00:00  182.88     198
 4          23780        Aaron Hughes  1979-11-08 00:00:00  182.88     154,
    player_api_id                 date  overall_rating preferred_foot  \
 0         505942  2016-02-18 00:00:00            67.0          right   
 1         505942  2015-11-19 00:00:00            67.0          right   
 2         505942  2015-09-21 00:00:00            62.0          right   
 3         505942  2015-03-20 00:00:00            61.0          right   
 4         505942  2007-02-22 00:00:00            61.0          right   
 
   attacking_work_rate defensive_work_rate  ball_control  dribbling  finishing  \
 0              medium     

# 4) CAST de fechas y filtrado por temporada 2015-2016
Convertimos `attr.date` a datetime (CAST) y filtramos el rango de la temporada.

In [5]:
attr['date'] = pd.to_datetime(attr['date'])  # CAST: texto -> fecha
season = attr[(attr['date'] >= '2015-08-01') & (attr['date'] <= '2016-07-31')].copy()

# Separar columnas numéricas y de texto
num_cols = [c for c in season.select_dtypes(include=['number']).columns if c != 'player_api_id']
text_cols = [c for c in season.columns if c not in num_cols + ['player_api_id','date']]

# Agregación por jugador: MEDIANA para numéricas y MODA para texto
agg = {c: 'median' for c in num_cols}
for c in text_cols:
    agg[c] = pd.Series.mode

attr_agg = season.groupby('player_api_id', as_index=False).agg(agg)

# Resolver posibles listas devueltas por mode() tomando el primer elemento
for c in text_cols:
    if c in attr_agg.columns:
        attr_agg[c] = attr_agg[c].apply(lambda x: x[0] if isinstance(x,(list,pd.Series,np.ndarray)) and len(x)>0 else x)

attr_agg.head()

,player_api_id,overall_rating,ball_control,dribbling,finishing,free_kick_accuracy,heading_accuracy,short_passing,shot_power,penalties,...,sprint_speed,agility,stamina,jumping,aggression,gk_diving,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,2752,72.0,57.0,41.0,40.0,19.0,82.0,59.0,60.0,38.0,...,39.0,34.0,62.0,55.0,84.0,12.0,16.0,right,medium,medium
1,2768,75.0,63.0,44.0,45.0,22.0,72.0,75.0,41.0,16.0,...,53.0,58.0,51.0,79.0,65.0,12.0,10.0,left,medium,medium
2,2796,68.0,19.0,14.0,12.0,11.0,17.0,23.0,23.0,20.0,...,52.0,63.0,34.0,83.0,28.0,81.0,75.0,right,medium,medium
3,2802,76.0,81.0,85.0,71.0,77.0,53.0,70.0,76.0,72.0,...,77.0,84.0,59.0,67.0,54.0,13.0,7.0,right,high,medium
4,2805,76.0,73.0,59.0,44.0,58.0,57.0,79.0,80.0,51.0,...,63.0,64.0,77.0,74.0,82.0,11.0,15.0,right,medium,medium


# 5) Completar jugadores sin registros en la temporada
Si faltan jugadores, tomamos su último registro hasta 2016-07-31 y lo añadimos.

In [6]:
players_with_season = set(attr_agg['player_api_id'])
attr_upto = attr[attr['date'] <= '2016-07-31'].copy()
missing = attr_upto[~attr_upto['player_api_id'].isin(players_with_season)]

cols_attr = num_cols + text_cols
if len(missing) > 0:
    last_missing = missing.sort_values(['player_api_id','date'], ascending=[True, False]).drop_duplicates('player_api_id')
    attr_agg = pd.concat([attr_agg, last_missing[['player_api_id'] + cols_attr]], ignore_index=True)

attr_agg.shape

(11060, 21)

# 6) Construir puente jugador-equipo-liga a partir de Match (melt)
Usamos CAST de fechas, filtro por 2016, melt de columnas de jugadores y determinamos el team_id según home/away.

In [7]:
match['date'] = pd.to_datetime(match['date'])  # CAST fecha
y2016 = match[match['date'].dt.year == 2016].copy()

player_cols = [c for c in y2016.columns if 'player' in c]
melted = pd.melt(
    y2016,
    id_vars=['match_api_id','date','home_team_api_id','away_team_api_id','league_id'],
    value_vars=player_cols,
    value_name='player_api_id'
).dropna(subset=['player_api_id'])

melted['player_api_id'] = melted['player_api_id'].astype(int)  # CAST int clave

last_game = (melted.sort_values(['player_api_id','date'], ascending=[True, False])
                    .drop_duplicates('player_api_id').copy())

def team_from_var(row):
    return row['home_team_api_id'] if 'home_player' in row['variable'] else row['away_team_api_id']

last_game['team_api_id'] = last_game.apply(team_from_var, axis=1)
bridge = last_game[['player_api_id','team_api_id','league_id']]
bridge.head()

,player_api_id,team_api_id,league_id
24174,2752,7878,21518
24219,2768,8370,21518
16209,2802,8370,21518
11315,2805,7878,21518
10745,2973,9803,13274


# 7) Enriquecer atributos con Team, League, Country (MERGE) y limpiar nulos
Realizamos MERGE sucesivos y eliminamos filas sin `team_long_name` (DROPNA).

In [8]:
enriched = attr_agg.merge(bridge, on='player_api_id', how='left')
enriched = enriched.merge(team[['team_api_id','team_long_name']], on='team_api_id', how='left')
enriched = enriched.merge(league[['id','league_name','country_id']].rename(columns={'id':'league_id'}), on='league_id', how='left')
enriched = enriched.merge(country[['id','country_name']].rename(columns={'id':'country_id'}), on='country_id', how='left')

before = enriched.shape[0]
enriched.dropna(subset=['team_long_name'], inplace=True)  # DROPNA: registros sin equipo
print('filas eliminadas por NaN en team_long_name:', before - enriched.shape[0])

enriched.head()

filas eliminadas por NaN en team_long_name: 7093


,player_api_id,overall_rating,ball_control,dribbling,finishing,free_kick_accuracy,heading_accuracy,short_passing,shot_power,penalties,...,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate,team_api_id,league_id,team_long_name,league_name,country_id,country_name
0,2752,72.0,57.0,41.0,40.0,19.0,82.0,59.0,60.0,38.0,...,16.0,right,medium,medium,7878.0,21518.0,Granada CF,Spain LIGA BBVA,21518.0,Spain
1,2768,75.0,63.0,44.0,45.0,22.0,72.0,75.0,41.0,16.0,...,10.0,left,medium,medium,8370.0,21518.0,Rayo Vallecano,Spain LIGA BBVA,21518.0,Spain
3,2802,76.0,81.0,85.0,71.0,77.0,53.0,70.0,76.0,72.0,...,7.0,right,high,medium,8370.0,21518.0,Rayo Vallecano,Spain LIGA BBVA,21518.0,Spain
4,2805,76.0,73.0,59.0,44.0,58.0,57.0,79.0,80.0,51.0,...,15.0,right,medium,medium,7878.0,21518.0,Granada CF,Spain LIGA BBVA,21518.0,Spain
6,2973,68.5,74.0,62.0,57.0,64.0,62.0,72.0,71.0,54.0,...,6.0,right,medium,medium,9803.0,13274.0,Roda JC Kerkrade,Netherlands Eredivisie,13274.0,Netherlands


# 8) Normalización de tipos y NaN en estadísticas
COERCE numéricos, FILLNA con mediana y CAST a int; texto con FILLNA a moda/'unknown'.

In [9]:
exclude = ['player_api_id','date','team_api_id','league_id','country_id','team_long_name','league_name','country_name']
stat_cols = [c for c in enriched.columns if c not in exclude]
text_stat_cols = ['preferred_foot','attacking_work_rate','defensive_work_rate']
num_stat_cols = [c for c in stat_cols if c not in text_stat_cols]

clean = enriched.copy()
for c in num_stat_cols:
    clean[c] = pd.to_numeric(clean[c], errors='coerce')  # COERCE -> NaN
    clean[c] = clean[c].fillna(clean[c].median())        # FILLNA mediana
    clean[c] = clean[c].astype(int)                      # CAST a int

for c in text_stat_cols:
    if c in clean.columns:
        moda = clean[c].mode()
        default = moda[0] if len(moda) > 0 else 'unknown'
        clean[c] = clean[c].fillna(default)              # FILLNA texto

clean[num_stat_cols].dtypes.head(10)

overall_rating        int64
ball_control          int64
dribbling             int64
finishing             int64
free_kick_accuracy    int64
heading_accuracy      int64
short_passing         int64
shot_power            int64
penalties             int64
acceleration          int64
dtype: object

# 9) Merge final con Player y exportación a CSV
Unimos por `player_api_id`, eliminamos IDs auxiliares y guardamos en UTF-8.

In [10]:
final = pd.merge(player, clean, on='player_api_id', how='inner')
final.drop(columns=['team_api_id','league_id','country_id'], inplace=True, errors='ignore')
final.to_csv(CSV_PATH, index=False, encoding='utf-8')
print('CSV guardado en:', CSV_PATH)
final.head()

CSV guardado en: C:\Users\Joaquim\OneDrive\UPGRADE\PROYECTO1_Soccer\data\data.csv


,player_api_id,player_name,birthday,height,weight,overall_rating,ball_control,dribbling,finishing,free_kick_accuracy,...,jumping,aggression,gk_diving,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate,team_long_name,league_name,country_name
0,155782,Aaron Cresswell,1989-12-15 00:00:00,170.18,146,73,70,71,51,69,...,84,67,14,12,left,high,medium,West Ham United,England Premier League,England
1,27316,Aaron Hunt,1986-09-04 00:00:00,182.88,161,77,79,78,72,76,...,65,59,15,15,left,medium,medium,Hamburger SV,Germany 1. Bundesliga,Germany
2,30895,Aaron Lennon,1987-04-16 00:00:00,165.10,139,77,81,84,66,55,...,71,57,14,11,right,high,medium,Everton,England Premier League,England
3,528212,Aaron Lennox,1993-02-19 00:00:00,190.50,181,48,22,15,15,18,...,38,21,53,53,right,medium,medium,Aberdeen,Scotland Premier League,Scotland
4,101042,Aaron Meijers,1987-10-28 00:00:00,175.26,170,69,74,70,42,67,...,70,73,7,15,left,medium,medium,ADO Den Haag,Netherlands Eredivisie,Netherlands
